In [13]:
from pathlib import Path 
import sys

REPO_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
sys.path.insert(0, str(REPO_ROOT))

import torch
from src.model import Model
from src.train_utils import (
    set_backbone_trainable,
    build_criterion,
    build_optimizer,
    train_step,
    eval_step,
)

In [14]:
# Ensuring the feature classifier is pass-through right now
m = Model()
print(m.backbone.fc)

Identity()


In [15]:
# Checking our forward pass is working
batch_size = 4
images = torch.randn(batch_size, 3, 224, 224)
with torch.no_grad():
    logits = m(images)

print(logits.shape)

torch.Size([4, 101])


In [16]:
# ensuring that we are freezing the ResNet50 correctly, should return true
set_backbone_trainable(m, False)
all(not p.requires_grad for p in m.backbone.parameters())

True

In [17]:
labels = torch.randint(0, 101, (batch_size,)) #fake labels
# creating criterion and optimizer
criterion = build_criterion()
optimizer = build_optimizer(m, 1e-3, 1e-4)

# single training step
loss, acc = train_step(m, images, labels, criterion, optimizer)
print(loss, acc)


4.643003940582275 0.0


In [18]:
#single eval step
val_loss, val_acc = eval_step(m, images, labels, criterion)

print(val_loss, val_acc)

4.356180667877197 0.25
